In [1]:
import numpy as np 

### 1A. Query data directly from PDB 

**Search query**

Getting all PDB IDs where the polymer type is "Protein" and where a bound ligand exists. 

In [1]:
from rcsbapi.search import search_attributes as attrs
# To see possible attributes, do attrs.__dict__ 

In [137]:
query = (
    (attrs.entity_poly.rcsb_entity_polymer_type == "Protein") & 
    (attrs.rcsb_ligand_neighbors.ligand_entity_id.exists()) & 
    (attrs.rcsb_binding_affinity.value.exists()) 
    # (attrs.rcsb_nonpolymer_entity.pdbx_number_of_molecules > 0) &
    # (attrs.rcsb_binding_affinity.value >= 0) & 
    # (attrs.rcsb_binding_affinity.value < ...) & 
    # (attrs.rcsb_nonpolymer_entity.formula_weight > 150) 
)
pdb_ids = list(query())
print(len(pdb_ids), "protein–ligand complexes")

14866 protein–ligand complexes


**Data query**

Getting the data for these PDB entries. For now, let's only get the Ligand IDs for now.  

In [84]:
from rcsbapi.data import DataQuery

In [139]:
test_pdb_ids = pdb_ids[:2]
test_pdb_ids

['10GS', '11GS']

In [138]:
query = DataQuery(
    input_type="entries",
    input_ids=test_pdb_ids,
    return_data_list=[
        "rcsb_id",
        "nonpolymer_entities.nonpolymer_comp.chem_comp.id"
    ]
)
protein_ligand_data = query.exec()
print(len(protein_ligand_data["data"]["entries"]), "protein–ligand complexes")

2 protein–ligand complexes


In [118]:
# Example protein–ligand complex record: 
protein_ligand_data["data"]["entries"][0]

{'rcsb_id': '10GS', 'nonpolymer_entities': [{'nonpolymer_comp': {'chem_comp': {'id': 'VWW'}}}, {'nonpolymer_comp': {'chem_comp': {'id': 'MES'}}}]}

In [125]:
# Average number of ligands per protein–ligand complex: 
np.mean([
    len(record["nonpolymer_entities"]) 
    for record in protein_ligand_complexes["data"]["entries"]
])

# If most protein–ligand complexes have more than 1 ligand, that could be a problem 
# as I don't know if our approach can handle this 

2.5

Now that we have the protein–ligand complexes, we still need to fetch data on those ligands, including their SMILES strings: 

In [149]:
# List of unique ligand IDs from above protein–ligand data: 
ligand_ids = list(set(
    ligand["nonpolymer_comp"]["chem_comp"]["id"] 
    for protein_ligand in protein_ligand_data["data"]["entries"] 
    for ligand in protein_ligand["nonpolymer_entities"]
))

In [150]:
test_ligand_ids = ligand_ids[:2]
test_ligand_ids

['VWW', 'GSH']

In [135]:
# For attributes, e.g. go to https://www.rcsb.org/ligand/VWW, click "Data API", which opens a GraphiQL UI, then run it. 

query = DataQuery(
    input_type="chem_comps",
    input_ids=test_ligand_ids,
    return_data_list=[
        "chem_comp.id",
        "chem_comp.type",
        "chem_comp.name",
        "chem_comp.formula",
        "rcsb_chem_comp_descriptor.SMILES",
        # Not sure if these might also come in handy: 
        "rcsb_chem_comp_info.atom_count", 
        "rcsb_chem_comp_info.bond_count", 
        "pdbx_chem_comp_identifier",
        "rcsb_chem_comp_related",
        "drugbank.drugbank_info.drugbank_id",
        "drugbank.durgbank_target.name",
        "drugbank.durgbank_target.interaction_type",
    ]
)
ligand_data = query.exec()
print(len(ligand_data["data"]["chem_comps"]), "ligands")

2 ligands


### 1B. Use Huggingface dataset `pdb_protein_ligand_complexes`

Download from here https://huggingface.co/datasets/jglaser/pdb_protein_ligand_complexes  
and put in `data/` directory.  
I also renamed them to: `pdb_protein_ligand_full_train.p` and `pdb_protein_ligand_full_test.p`  
I removed unnecessry columns and saved the new datasets to: `pdb_protein_ligand_train.p` and `pdb_protein_ligand_test.p`  
which are much smaller. 

In [2]:
import pandas as pd

In [34]:
# protein_ligand_train = pd.read_pickle("data/pdb_protein_ligand_full_train.p")
# protein_ligand_train[["pdb_id", "lig_id", "seq", "smiles"]].to_pickle("data/pdb_protein_ligand_train.p")
protein_ligand_train = pd.read_pickle("data/pdb_protein_ligand_train.p")

# protein_ligand_test = pd.read_pickle("data/pdb_protein_ligand_full_test.p")
# protein_ligand_test[["pdb_id", "lig_id", "seq", "smiles"]].to_pickle("data/pdb_protein_ligand_test.p")
protein_ligand_test = pd.read_pickle("data/pdb_protein_ligand_test.p")

In [36]:
protein_ligand_test.iloc[:10]

,pdb_id,lig_id,seq,smiles
0,7k38,VTY,MGIVEEAHNVKVLGTGSRFIVLAHGFGTDQSVWKHLVPHLLEEFRV...,CC1=C[C@@H](O)OC1=O
1,6prt,OWA,SNPPPPETSNPNKPKRQTNQLQYLLRVVLKTLWKHQFAWPFQQPVD...,COC(=O)C[C@H]1CC(=O)N(C)C1
2,4lxx,FNF,GHMIKICIAGKNNIAVNSLQFILKNYFEADQIVVIPNKNDKGIDSW...,Cc1cn([C@H]2C[C@H](O)[C@@H](COP(=O)(O)OP(=O)(O...
3,4lxx,FON,GHMIKICIAGKNNIAVNSLQFILKNYFEADQIVVIPNKNDKGIDSW...,Nc1nc(=O)c2c([nH]1)NC[C@@H](CNc1ccc(C(=O)N[C@@...
4,7bp1,CAQ,MLGKVALEEAFALPRHKERTRWWAGLFAIDPDKHAAEINDITEQRI...,Oc1ccccc1O
5,4ibj,1D9,PDISAKDLRNIMYDHLPGFGTAFHQLVQVICKLGKDSNSLDIIHAE...,O=C(O)c1cccc(N2C(=O)C(O)=C(C(=O)c3cccc(C(F)(F)...
6,1psa,0ZL,IGDEPLENYLDTEYFGTIGIGTPAQDFTVIFDTGSSNLWVPSVYCS...,CCOC(=O)N[C@@H](CC(C)C)C(=O)N[C@@H](CC(C)C)C(=...
7,5ag7,XXL,AHAFWSTQPVPQTEDETEKIVFAGPMDEPKTVADIPEEPYPIASTF...,CCOC(=O)CN1C(=O)COc2ccccc21
8,1kdr,CAR,AIAPVITIDGPSGAGKGTLCKAMAEALQWHLLDSGAIYRVLALAAL...,Nc1ccn([C@@H]2O[C@H](COP(=O)(O)O)[C@@H](O)[C@@...
9,1v0c,KNC,DSVTLRLMTEHDLAMLYEWLNRSHIVEWWGARPTLADVQEQYLPSV...,N[C@H]1[C@H](O)[C@@H](CO)O[C@H](O[C@@H]2[C@@H]...


### 2. Apply SynFormer to the SMILES strings to generate the dataset of synthesis tokens 

<div class="alert alert-warning">  
Unfortunately, this code doesn't work for me if I simply install/import <b>synformer</b><br/>
Instead, I cloned the repo (https://github.com/wenhao-gao/synformer) and put this notebook in there<br/>
Then in the virtual environment, I did "pip install -e ." to install the local version<br/>
I also had to fix something in that local version:<br/>
In synformer/sampler/analog/cli.py, line 4, change<br/> 
<pre style="background-color:lightgray">
from parallel import run_parallel_sampling
</pre>
to this:<br/>
<pre style="background-color:lightgray">
from synformer.sampler.analog.parallel import run_parallel_sampling
</pre>
</div>

In [ ]:
# I want a dataset (lig_id, original_smiles, projected_smiles), and then possibly multiple projections 
# per lig_id entry, so (lig_id, original_smiles, projected_smiles, projection_id) 
# But besides the SMILES, we need all 3 outputs from SynFormer: token type, reaction token, reactant token 
# If I remember correctly, there is a vocabulary of reaction tokens, so it would be enough to store the reaction token IDs,
# which we can look up in the 
# I'm not sure if we can do the same with the reactants, as there is no vocabulary. But from the ID, perhaps we can
# get the fingerprint and then at will we can pass it through the embedding layer to get the embedding vector 
# Because if we store the entire embedding vector for each reactant in all examples, that dataset would be unnecessarily big
# Ideally, we just keep the IDs 
# We want the ground-truth embeddings anyway, not the predicted embeddings which would then be Nearest-Neighbor'ed to the ground truth embedding 
# So now the dataset would look like this: 

# (lig_id, original_smiles, projected_smiles, projection_id, token_types_tensor, reactions_tensor, reactants_tensor) 

# the tensors all seem to have a fixed sequence length of 24 
# I could also simply store all non-END and non-START tokens, and then during the prediction I prepend/append them, as they're always the same 

# optionally, I could also store the similarity score between the original and projected molecules 

The following code is partly based on `scripts/sample_naive.py` form the Synformer repo 

In [ ]:
# Imports from scripts/sample_naive.py: 
import pathlib
import pickle
import torch
from omegaconf import OmegaConf
from synformer.chem.fpindex import FingerprintIndex
from synformer.chem.matrix import ReactantReactionMatrix
from synformer.chem.mol import Molecule
from synformer.models.synformer import Synformer

# My own imports:
import pandas as pd 
from scripts.sample_naive import sample_naive, load_model, featurize_smiles 
# from synformer.models.synformer import draw_generation_results
from synformer.data.common import TokenType
from datetime import datetime 

In [ ]:
MODEL_PATH = "data/trained_weights/sf_ed_default.ckpt"
CONFIG_PATH = None 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model, fpindex, rxn_matrix = load_model(MODEL_PATH, CONFIG_PATH, DEVICE)

In [ ]:
def get_synthetic_pathway(smiles, lig_id=None, repeat=1):
    data = []
    mol, feat = featurize_smiles(smiles, DEVICE, repeat=repeat)
    with torch.inference_mode():wa
        result = model.generate_without_stack(
            feat,
            rxn_matrix=rxn_matrix,
            fpindex=fpindex,
            temperature_token=1.0,
            temperature_reactant=0.1,
            temperature_reaction=1.0,
        )
        ll = model.get_log_likelihood(
            code=result.code,
            code_padding_mask=result.code_padding_mask,
            token_types=result.token_types,
            rxn_indices=result.rxn_indices,
            reactant_fps=result.reactant_fps,
            token_padding_mask=result.token_padding_mask,
        )
    stacks = result.build() 
    for i, stack in enumerate(stacks):
        # Only those sequences with stack depth of 1 (i.e. applying the building blocks and reactions leads to 1 final molecule) 
        # are considered valid results!! 
        if stack.get_stack_depth() == 1:
            analog_mol = stack.get_one_top()
            sim = analog_mol.sim(mol)
            # TODO: perhaps only continue if similarity score sufficiently high? 
            token_types = result.token_types[i]
            # Location of first END token (we only need to store tokens up until this point): 
            end_id = token_types.tolist().index(0)
            token_types = token_types[:end_id].tolist()
            rxn_indices = result.rxn_indices[i,:end_id].tolist()
            reactant_indices = result.reactant_indices[i,:end_id].tolist()
            data.append([
                lig_id,
                smiles, 
                analog_mol.smiles, 
                round(sim, 4), 
                token_types,
                rxn_indices,
                reactant_indices
            ])
    return data

Apply to dataset:

In [47]:
protein_ligand_train = pd.read_pickle("data/pdb_protein_ligand_train.p")
ligands_train = protein_ligand_train[["lig_id", "smiles"]].drop_duplicates()
del protein_ligand_train
print(ligands_train.shape)

protein_ligand_test = pd.read_pickle("data/pdb_protein_ligand_test.p")
ligands_test = protein_ligand_test[["lig_id", "smiles"]].drop_duplicates()
del protein_ligand_test
print(ligands_test.shape)

(21096, 2)
(3148, 2)


In [ ]:
%%time 

data = {}
repeat = 1

# for ligands, dataset_name in [(ligands_test, "test"), (ligands_train, "train")]:
ligands = ligands_test
dataset_name = "test" 
data[dataset_name] = []

for i, row in ligands.iterrows():
    try:
        lig_id = row["lig_id"].item() 
        smiles = row["smiles"]
        print(i, lig_id, smiles)
        records = get_synthetic_pathway(smiles, lig_id=lig_id, repeat=repeat)
        for record in records:
            if record not in data[dataset_name]:
                data[dataset_name].append(record)
    except Exception as e:
        # raise e 
        print(f"Error processing SMILES {i} ({e})")
pickle.dump(
    data[dataset_name], 
    open(f"data/synformer_ligands_{dataset_name}_{datetime.now().strftime('%Y-%m-%d %H-%M-%S')}.pkl", "wb")
)

In [ ]:
columns = ["lig_id", "smiles_original", "smiles_proj", "similarity", "token_types", "rxn_indices", "reactant_indices"]

df = pd.DataFrame(data, columns=columns)
df

Just for convenience: reconstructing the sequence of building blocks and reactions: 

In [ ]:
def reconstruct_pathway(token_types, rxn_indices, reactant_indices):
    """
    From a list of token types, reaction indices and reactant indices, construct the synthetic pathway
    E.g. [1, 3], [0, 99], [0, 10]
         This would be mapped to [START, B10, END]
         At seq index 0, we have token type 1, which corresponds to the START token, so that's the first token 
         At seq index 1, we have token type 3, which corresponds to a REACTANT token, which we fetch from reactant_indices
                         and in this case is token ID 10, so the token is "B10"
         That's all of them. 
         At the end, I append an END token.
    Token types:
      0: END token (also used for padding, following the actual END token) 
      1: START token 
      2: REACTION
      3: REACTANT
    """
    pathway = []
    for i, token_type in enumerate(token_types):
        match token_type:
            case 1:
                token = "START"
            case 2:
                token = f"R{rxn_indices[i]}"
            case 3:
                token = f"B{reactant_indices[i]}"
            case _:
                token = None 
        pathway.append(token)
    pathway.append("END")
    return pathway

reconstruct_pathway([1, 3], [0, 85], [-1, 32101])

In [ ]:
for _, row in df.iloc[:10].iterrows():
    print(reconstruct_pathway(row["token_types"], row["rxn_indices"], row["reactant_indices"]))